In [1]:
!pip install pandas_datareader
import pandas as pd
import datetime as dt
import pandas_datareader.data as web
from datetime import datetime
from bokeh.plotting import figure
from bokeh.io import output_notebook,show
output_notebook()
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource,SingleIntervalTicker,Range1d,LinearAxis
from bokeh.models import LabelSet, ColumnDataSource, Range1d, NumeralTickFormatter


from bokeh.transform import factor_cmap
from bokeh.palettes import Category20c


%load_ext bokeh
start_time = dt.datetime(2010, 1, 1)
end_time = dt.datetime(2022, 2, 1)


from bokeh.palettes import Category20



Loading BokehJS ...

The bokeh module is not an IPython extension.


In [2]:
# country_codes=['HUN', 'CZE', 'MEX', 'POL', 'ZAF', 'IND', 'IDN', 'BRA', 'CHN']
country_codes=[ 'IND','BRA', 'IDN', 'CHN']


In [3]:
df_rates=pd.read_csv('Long-term-interest-rates.csv')
df_rates = df_rates[df_rates['LOCATION'].isin(country_codes)]
df_rates=df_rates[df_rates['TIME']>'2013-Q4']

df_rates = df_rates[df_rates['TIME'].str.contains('Q1|Q4')]


In [4]:
# https://data.oecd.org/interest/long-term-interest-rates.htm#indicator-chart
def rates_bop(df_rates,y_label,bop_fdi):
    
    if(bop_fdi!=1):
        n_countries = len(df_rates['LOCATION'].unique())
        line_colors = Category20[n_countries]    
        p = figure(width=1500, height=700, x_range=df_rates['TIME'].unique(), x_axis_label='Date', y_axis_label=y_label)

    else:
        p = figure(title="India current account and FDI Inflows ",width=1500, height=700, x_range=df_rates['TIME'].unique(), x_axis_label='Date', y_axis_label=y_label)
    
    locations = df_rates['LOCATION'].unique()
    
    # Create a multiline for each country
    for i, location in enumerate(locations):
        location_data = df_rates[df_rates['LOCATION'] == location]
        if(bop_fdi):
            p.vbar(x='TIME', top='Value_x',  fill_alpha=0.8, color='blue',source=ColumnDataSource(location_data), legend_label='Current Account',width = 0.5, bottom = 0)
            p.line(x='TIME', y='Value_y', source=ColumnDataSource(location_data), legend_label='FDI Inflows', line_width=2,line_color='green', muted_alpha=0.2)
        else:
            p.line(x='TIME', y='Value', source=ColumnDataSource(location_data), legend_label=f'Country: {location}', line_width=2,line_color=line_colors[i], muted_alpha=0.2)
            p.legend.title = 'Countries'
    
    p.xaxis.major_label_orientation = 45
    
    p.legend.click_policy = "mute" 
    p.legend.location = "top_left"

    
    show(p)


In [5]:
rates_bop(df_rates,'Long term rates (%)',0)

In [6]:
balanceofpayments=pd.read_csv('BalanceOfPayments.csv')
country_codes=['IND','IDN','BRA']

bop = balanceofpayments[balanceofpayments['LOCATION'].isin(country_codes)]
bop=bop[bop['MEASURE']=='MLN_USD']
bop=bop.sort_values('TIME')
# bop=bop[(df_rates_forecast['TIME']>'2016') &(bop['TIME']<'2024')]
bop = bop[bop['TIME'].str.contains('Q1|Q2|Q3|Q4')]

rates_bop(bop,'Balance Of Payments',0)

In [7]:
fdi_inflows=pd.read_csv('FDI_Inflows.csv')
country_codes=['IND','IDN','BRA']

fdi = fdi_inflows[fdi_inflows['LOCATION'].isin(country_codes)]
fdi=fdi[fdi['SUBJECT']=='INWARD']

fdi=fdi[fdi['MEASURE']=='MLN_USD']
fdi=fdi.sort_values('TIME')
# bop=bop[(df_rates_forecast['TIME']>'2016') &(bop['TIME']<'2024')]
fdi = fdi[fdi['TIME'].str.contains('Q1|Q2|Q3|Q4')]

rates_bop(fdi,'FDI Inflows',0)

In [8]:
common_cols=['LOCATION','TIME']
fdi_inflows=pd.read_csv('FDI_Inflows.csv')
country_codes=['IND']

fdi = fdi_inflows[fdi_inflows['LOCATION'].isin(country_codes)]
fdi=fdi[fdi['SUBJECT']=='INWARD']

fdi=fdi[fdi['MEASURE']=='MLN_USD']
fdi=fdi.sort_values('TIME')
# bop=bop[(df_rates_forecast['TIME']>'2016') &(bop['TIME']<'2024')]
fdi = fdi[fdi['TIME'].str.contains('Q1|Q2|Q3|Q4')]

In [9]:
bop_fdi=pd.merge(bop,fdi,on=common_cols,how='inner')
bop_fdi['pct']=(bop_fdi.Value_y/bop_fdi.Value_x)


In [10]:
rates_bop(bop_fdi,'Curent Account & FDI (USD millions)',1)

In [11]:
df=bop_fdi
df=df[(df['TIME']>'2020')]

p = figure(width=1500, height=700, x_range=df['TIME'].unique(), x_axis_label='Date', y_axis_label='FDI as a % of Curent Account')
locations = df['LOCATION'].unique()

for i, location in enumerate(locations):
        location_data = df[df['LOCATION'] == location]
p.line(x='TIME', y='pct', source=ColumnDataSource(location_data), legend_label='FDI as a % of Curent Account', line_width=2,line_color='green', muted_alpha=0.2)
p.xaxis.major_label_orientation = 45
p.legend.click_policy = "mute" 
p.legend.location = "top_left"
show(p)


In [12]:
# https://data.oecd.org/interest/long-term-interest-rates-forecast.htm#indicator-chart

df_rates_forecast=pd.read_csv('Long-term-interest-rates-forecast.csv')
country_codes=['HUN', 'CZE', 'MEX', 'POL', 'ZAF', 'IND', 'IDN', 'BRA', 'CHN']

df_rates_forecast = df_rates_forecast[df_rates_forecast['LOCATION'].isin(country_codes)]
df_rates_forecast=df_rates_forecast.sort_values('TIME')
df_rates_forecast=df_rates_forecast[(df_rates_forecast['TIME']>'2016') &(df_rates_forecast['TIME']<'2024')]
# df_rates_forecast = df_rates_forecast[df_rates_forecast['TIME'].str.contains('Q1|Q4')]
df_rates_forecast.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
5520,POL,LTINTFORECAST,TOT,PC_PA,Q,2016-Q1,2.972281,NaN
8018,ZAF,LTINTFORECAST,TOT,PC_PA,Q,2016-Q1,9.373135,NaN
2665,CZE,LTINTFORECAST,TOT,PC_PA,Q,2016-Q1,0.476667,NaN
4685,MEX,LTINTFORECAST,TOT,PC_PA,Q,2016-Q1,6.110000,NaN
7329,IND,LTINTFORECAST,TOT,PC_PA,Q,2016-Q1,7.519900,NaN


In [13]:
rates_bop(df_rates_forecast,'Rate Forecast %',0)

In [14]:
# https://stats.oecd.org/Index.aspx?QueryId=92129#
# df_flows = web.DataReader('FDI_AGGR_SUMM', 'oecd', start_time, end_time) #Data source is migrating hence API is not usable 
# df_India_inw=df_flows[df_flows.index=='India']['US Dollar']['Directional principle: Inward']#['FDI financial flows - Debt']
df=pd.read_csv('FDI_AGGR_SUMM.csv')

def format(df):
    df=df[(df['Reporting country']=='India')  & (df['Type of FDI']=='FDI financial flows - Debt')& (df['Unit']=='US Dollar')]
    df = df[df['Time'].str.contains('Q1|Q2|Q3|Q4')]


    # Extract the quarter and year into separate columns
    df['Year'] = df['TIME'].str.split('-').str[0]
    df['Quarter'] = df['TIME'].str.split('-').str[1]

    # Define a custom sort order for quarters
    quarter_order = ['Q1', 'Q2', 'Q3', 'Q4']

    # Sort the DataFrame based on year and quarter
    df = df.sort_values(by=['Year', 'Quarter'], key=lambda x: x.map({q: i for i, q in enumerate(quarter_order)}))

    # Reset the index if needed
    df.reset_index(drop=True, inplace=True)

    df.index=df['TIME']
    df['Year']=df['Year'].astype(int)
    df=df[(df['Year'])>2015]
    df = df.drop_duplicates()
    return(df)
df_flows=format(df)


In [15]:

df_India_inw=df_flows[df_flows['Measurement principle']=='Directional principle: Inward']
df_India_out=df_flows[df_flows['Measurement principle']=='Directional principle: Outward']


In [16]:
def India(df_inw,df_out,column):
    
    df=pd.merge(df_inw[column],df_out[column],how='inner',left_index=True,right_index=True)
    df = df.drop_duplicates()
    categories = df.index.tolist()


    p = figure(title="India FDI financial flows - Debt", x_axis_label='Year', y_axis_label='Flows (USD Millions)',x_range=categories,width=1500)
    x=df.index
    y_inw=df[column+'_x']
    y_out=df[column+'_y']
    

    p.line(x, y_inw, legend_label=column+' inward', line_width=2,color='green')
    p.line(x, y_out, legend_label=column+' outward', line_width=2,color='red')
    
    # p.xaxis.ticker.desired_num_ticks=15

    p.xaxis.major_label_orientation = 45
    p.legend.location = "top_left"


    show(p)

In [17]:
India(df_India_inw,df_India_out,'Value')
# Data Source: https://stats.oecd.org/Index.aspx?QueryId=92129#

In [18]:
!pip install PublicDataReader --upgrade
!pip install fredapi

# https://fred.stlouisfed.org/series/INDBCABP6USD
from fredapi import Fred
fred = Fred(api_key='46638c506833ac8e65a7798cf2c5315e')

In [19]:
# data = fred.get_series('SP500')
# data = fred.get_series('DAAA')#inv grade bonds
def fred_data_func(tag,metric):
    df=pd.DataFrame(fred.get_series(tag),columns=[metric]) 
    return(df)

In [20]:
CPI = fred_data_func('INDCPIALLMINMEI','CPI')
BOP = fred_data_func('INDBCABP6USD','BOP')
GDP = fred_data_func('NGDPRNSAXDCINQ','GDP')


data1=pd.merge(CPI,BOP, left_index=True, right_index=True,how='inner')
fred_data=pd.merge(data1,GDP, left_index=True, right_index=True,how='inner')

fred_data.index.name='Date'
fred_data=fred_data[fred_data.index.year>2012]
fred_data['BOP']=fred_data['BOP']/1000000
fred_data

,CPI,BOP,GDP
Date,,,
2013-01-01,84.539369,NaN,24670800.0
2014-01-01,90.659866,NaN,25989000.0
2015-01-01,97.162894,NaN,27837300.0
2016-01-01,102.900861,-14351.000000,30367400.0
2017-01-01,104.813516,-48662.000000,32277300.0
2018-01-01,110.168951,-57183.000000,35160600.0
2019-01-01,117.437042,-57805.627897,37172100.0
2020-01-01,126.235257,-73539.005381,38266800.0
2021-01-01,130.228035,-80444.597061,39557800.0


In [21]:
from bokeh.layouts import column
import numpy as np

def fred_chart(df):
    
    # pandas dataframe
    x_column = "Date"
    y_column1 = "BOP"
    y_column2 = "CPI"
    y_column3 = "GDP"
    
    # Bokeh plot
    
    y_overlimit = 0 #0.05 # show y axis below and above y min and max value
    p = figure(x_axis_type='datetime',x_axis_label=x_column,
           y_axis_label=str(y_column1)+' (USD billion) ')#, tools="hover", tooltips="$name @y_column2: @$name")
    
    # FIRST AXIS
    p.line(df.index, df[y_column1], line_width=1, color="blue",legend_label=y_column1)
    # p.vbar(x=df.index, top=df[y_column1], width=1, legend_label=y_column1) #color="red",

    p.y_range = Range1d(
        df[y_column1].min()-1 ,#* (1 - y_overlimit), 
        df[y_column1].max()+1 #* (1 + y_overlimit)
    )
    
    # SECOND AXIS
    y_column2_range = y_column2 + "_range"
    p.extra_y_ranges = {
        y_column2_range: Range1d(
            start=df[y_column2].min()-10 ,#* (1 - y_overlimit),
            end=df[y_column2].max()+10 ,#* (1 + y_overlimit),
        )
    }
    p.add_layout(LinearAxis(y_range_name=y_column2_range,axis_label=y_column2), "right")
    
    p.line(
        df.index,
        df[y_column2],
        legend_label=y_column2,
        line_width=1,
        y_range_name=y_column2_range,
        color="green",
    )
    
    p.legend.location = "top_left"
    p.yaxis.formatter=NumeralTickFormatter(format=",000")

          
    show(p)



fred_chart(fred_data)

In [22]:
#10yr yields: INDIRLTLT01STM
India_Yield_10yr = fred_data_func('INDIRLTLT01STM','India 10yr_yield')
US_Yield_10yr = fred_data_func('DGS10','US 10yr_yield')
Yield_10yr=pd.merge(India_Yield_10yr,US_Yield_10yr, left_index=True, right_index=True,how='inner')
Yield_10yr=Yield_10yr.bfill()
# df.dropna()
Yield_10yr.index.name='Date'
Yield_10yr=Yield_10yr[Yield_10yr.index>'2018-01-01']


In [23]:
from bokeh.layouts import column
import numpy as np

def fred_chart(df):
    
    # pandas dataframe
    x_column = "Date"
    y_column1 = "India 10yr_yield"
    y_column2 = "US 10yr_yield"
    


    y_overlimit = 0 #0.05 # show y axis below and above y min and max value
    p = figure(x_axis_type='datetime',x_axis_label=x_column,
           y_axis_label=y_column1)#, tools="hover", tooltips="$name @y_column2: @$name")
    
    # FIRST AXIS
    p.line(df.index, df[y_column1], line_width=1, color="blue",legend_label=y_column1)

    p.y_range = Range1d(
        df[y_column1].min()-1 ,#* (1 - y_overlimit), 
        df[y_column1].max()+1 #* (1 + y_overlimit)
    )
    
    # SECOND AXIS
    y_column2_range = y_column2 + "_range"
    p.extra_y_ranges = {
        y_column2_range: Range1d(
            start=df[y_column2].min()-1 ,#* (1 - y_overlimit),
            end=df[y_column2].max()+1 ,#* (1 + y_overlimit),
        )
    }
    p.add_layout(LinearAxis(y_range_name=y_column2_range,axis_label=y_column2), "right")
    
    p.line(
        df.index,
        df[y_column2],
        legend_label=y_column2,
        line_width=1,
        y_range_name=y_column2_range,
        color="green",
    )

    
    p.legend.location = "top_left"

    
    show(p)

fred_chart(Yield_10yr)